In [103]:
import subprocess
import time
import random
import re

#Screen size
x_max = 1080 #px
y_max = 2640 #px

#GUI element locations
#Home
coords_home = [150, 2500]
coords_search_down = [315, 2500]
coords_search_up = [500, 200]

def tap(x, y):
    subprocess.run(["adb", "shell", "input", "tap", str(x), str(y)])

def swipe(x1, y1, x2, y2, duration=200):
    subprocess.run([
        "adb", "shell", "input", "swipe",
        str(x1), str(y1), str(x2), str(y2), str(duration)
    ])

def type_text(text):
    encoded = text.replace(" ", "%s")
    subprocess.run(["adb", "shell", "input", "text", encoded])

def tap_coords(coords):
    #tap with just putting coordinates vector
    tap(coords[0],coords[1])

def tap_coords_normal(coords,deviation):
    #tapper with normal distribution (or not normal i don't remembe)
    #has random element to tap
    tap_coords(coords+[deviation*(random.random()-0.5), deviation*(random.random()-0.5)])
    
def swipe_back():
    deviation = 100;
    x1 = 1000+deviation*(random.random()-0.5)
    y1 = 1500+deviation*(random.random()-0.5)
    x2 = 800+deviation*(random.random()-0.5)
    y2 = 1500+deviation*(random.random()-0.5)
    duration  = round(200 + 2*deviation*(random.random()-0.5))
    swipe(x1, y1, x2, y2, duration)

def skroll_normal(down = True):

    deviation = 100;
    x1 = 500+deviation*(random.random()-0.5)
    y1 = 1500+deviation*(random.random()-0.5)
    x2 = 500+deviation*(random.random()-0.5)
    y2 = 800+deviation*(random.random()-0.5)
    duration  = round(200 + 2*deviation*(random.random()-0.5))
    if down:
        swipe(x1, y1, x2, y2, duration)
    else:
        swipe(x1, y2, x2, y1, duration)

def get_current_app():
    cmd = 'adb shell "dumpsys activity recents | grep \\"Recent #0\\""'
    proc = subprocess.run(
        cmd,
        stdout=subprocess.PIPE, text=True,
        shell=True
    )
    out = proc.stdout.strip()
    
    if not out:
        return None

    # Extract the package name between com. and .android
    match = re.search(r'com\.([^.]+)\.android', out)
    if match:
        return match.group(1)  # This will return 'instagram' from your example
    return None

def abort_if_wrong_app():
    app = get_current_app()
    if app != 'instagram':
        print(f"[ABORT] Switched app detected: {app}")
        raise SystemExit

In [36]:
#Tap Home
tap(coords_home[0],coords_home[1])

In [50]:
tap_coords_normal(coords_home, 50)

In [52]:
tap_coords_normal(coords_search, 50)

In [69]:
skroll_normal()

In [109]:
# ---------------------------------------------------------
# UPDATED random_tap + slower timing + scroll-down-only
# ---------------------------------------------------------

random_words = [
    "apple", "chair", "planet", "python", "river", "coffee", "android",
    "music", "random", "science", "mountain", "game", "network"
]

def random_tap():
    # tap around the vertical middle ± 500px
    mid_x = x_max // 2
    mid_y = y_max // 2

    x = random.randint(mid_x - 200, mid_x + 200)     # horizontal tight area
    y = random.randint(mid_y - 500, mid_y + 500)     # ±500px vertically

    tap(x, y)

# Force scroll to be DOWN only
def skroll_normal_down_only():
    deviation = 100
    x1 = 500 + deviation*(random.random()-0.5)
    y1 = 1500 + deviation*(random.random()-0.5)
    x2 = 500 + deviation*(random.random()-0.5)
    y2 = 800 + deviation*(random.random()-0.5)
    duration = round(350 + 2*deviation*(random.random()-0.5))  # slower scroll
    swipe(x1, y1, x2, y2, duration)

def behavior_cycle(n_rounds=5):
    for i in range(n_rounds):
        abort_if_wrong_app()
        print(f"--- Cycle {i+1} ---")
        
        # Step 1: Always tap 'search_down'
        tap_coords_normal(coords_search_down, deviation=40)
        time.sleep(random.uniform(1.0, 1.4))

        # Random decision: Path A or Path B
        path = random.choice(["A", "B"])

        # ----------------------------------------------------------
        # PATH A: Press search_up → type random word → scroll around
        # ----------------------------------------------------------
        if path == "A":
            print("Path A: searching, typing, scrolling")
            abort_if_wrong_app()
            tap_coords_normal(coords_search_up, deviation=40)
            time.sleep(random.uniform(1.0, 1.4))

            # Random word
            word = random.choice(random_words)
            type_text(word)
            time.sleep(0.8)

            # Press search (you defined this manually)
            tap(1000, 2420)
            time.sleep(random.uniform(1.4, 2.0))

            # 3–6 downward scrolls (slower)
            for _ in range(random.randint(3, 6)):
                abort_if_wrong_app()
                skroll_normal_down_only()
                time.sleep(random.uniform(0.8, 1.4))

        # ----------------------------------------------------------
        # PATH B: Scroll randomly → taps → swipe back
        # ----------------------------------------------------------
        else:
            print("Path B: random scrolling & tapping")

            for _ in range(random.randint(3, 7)):
                # DOWN only now
                abort_if_wrong_app()
                skroll_normal_down_only()
                time.sleep(random.uniform(0.9, 1.5))

                # Middle-screen tap
                abort_if_wrong_app()
                random_tap()
                time.sleep(random.uniform(0.6, 1.2))

            swipe_back()
            time.sleep(1.0)

        time.sleep(random.uniform(1.0, 1.4))


In [113]:
# Run the algorithm
behavior_cycle(n_rounds=5)

--- Cycle 1 ---
Path B: random scrolling & tapping
--- Cycle 2 ---
Path B: random scrolling & tapping
--- Cycle 3 ---
Path B: random scrolling & tapping
--- Cycle 4 ---
Path B: random scrolling & tapping
[ABORT] Switched app detected: None


adb: no devices/emulators found


SystemExit: 

/home/arc/Projects/Nano_banana/banana_venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [85]:
def get_current_app():
    proc = subprocess.run(
        ["adb", "shell", "dumpsys activity recents | grep 'Recent #0'"],
        stdout=subprocess.PIPE, text=True, shell=False
    )
    return proc.stdout.strip()


In [95]:
import re
import subprocess

def get_current_app():
    cmd = 'adb shell "dumpsys activity recents | grep \\"Recent #0\\""'
    proc = subprocess.run(
        cmd,
        stdout=subprocess.PIPE, text=True,
        shell=True
    )
    out = proc.stdout.strip()
    
    if not out:
        return None

    # Extract the package name between com. and .android
    match = re.search(r'com\.([^.]+)\.android', out)
    if match:
        return match.group(1)  # This will return 'instagram' from your example
    return None



In [104]:
get_current_app()

In [108]:
abort_if_wrong_app()


[ABORT] Switched app detected: None


SystemExit: 